# Manipulating models

## Preparation

In [1]:
from cobra.io import read_sbml_model

In [2]:
model = read_sbml_model('data/e_coli_core.xml.gz')

## Making temporary changes to the model (5 + 2)

Usually one relies on making copies if objects need to be changed but the original state needs to be retained and that's we what we did previously with the `model_copy = model.copy()`. Unfortunately, metabolic models can be enormous and copying those in memory can take quite long.

In [3]:
%%time
copy_of_model = model.copy()

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 17 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [4]:
%%time
## avoid writing code like this...
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 2.19 s, sys: 64 ms, total: 2.26 s
Wall time: 2.25 s


For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the model such as knocking-out genes, reactions, adding or removing metabolites, reactions, etc. can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [5]:
%%time
## instead do this; no copying!
with model:
    for gene in model.genes:
        gene.knock_out()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.9 ms


Here, the `with model` statement starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before entering the context.

Changing flux bounds can, as indicated, also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effectively knocking out the reaction).

In [6]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  0.0
PGK's bounds outside the with statement
(-1000.0, 1000.0)


### Exercises

Convert the cell below to code and fill in the blanks

In [7]:
# is the PGM reaction essential?
with model:
    model.reactions.PGM.bounds = 0,0 # Setting bounds to (0,0) so it doesn't run
    trying =  model.optimize().objective_value # Optimizing the objective value (biomass production)
    print('growth without PGM is max', trying) # Since result is 0.0, PGM rxn is essential
        
    

growth without PGM is max 0.0


## Changing the medium (2 + 3)

One can access the medium condition using `model.medium`. The indicated bound is the effective upper uptake bound. 

In [7]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [11]:
mediums = model.medium
with model:
    mediums['EX_glc__D_e'] = 0
    mediums['EX_succ_e'] = 10
    model.medium = mediums
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.39756301542776384


Changing the carbon source to succinate led to a significant drop in growth rate.

### Exercise

Convert the cell below to code and fill in the blanks

In [ ]:
# what is the max growth rate when growing on fumarate instead of glucose


# Attempting a different medium with fumarate (Other_Carbon_Medium)
other_carbon_medium = model.medium

with model:
    other_carbon_medium['EX_glc__D_e'] = 0
    other_carbon_medium['EX_fum_e'] = 10
    model.medium = other_carbon_medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])


# Since method restores original model without making a direct copy of it, here is the /
# Original medium with glucose (Main_Medium)
Main_Medium = model.optimize()
print(Main_Medium.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.3707405207962852
0.8739215069684307


## Adding metabolites, reactions and pathways (10 + 10)

In [15]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [ ]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold. First we need to create a new metabolite, since gold is not yet part of *E. coli's* native metabolism.

In [17]:
gold = Metabolite(id='gold_c', compartment='c', name='GOLD')

Now, we're going to specify the reaction's stoichiometry.

In [18]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [19]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [ ]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [21]:
model.reactions.alchemy

Reaction identifier,alchemy
Name,
Memory address,0x1b08df9a020
Stoichiometry,h2o_c --> gold_c H2O --> GOLD
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's produce some gold then!

In [22]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

0.0

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [17]:
model.add_boundary(model.metabolites.gold_c, type='demand')

Reaction identifier,DM_gold_c
Name,GOLD demand
Memory address,0x07f3d95b28da0
Stoichiometry,gold_c --> GOLD -->
GPR,
Lower bound,0
Upper bound,1000.0


In [18]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

1000.0

Yes, much better!

### Exercise


Convert the cell below to code and fill in the blanks. Add a magic pathway that converts glucose directly to ATP. Does this lead to an increase in growth rate?

In [ ]:
# Trying to figure out metabolite identifiers
from cobra.io import read_sbml_model
model = read_sbml_model("data/e_coli_core.xml.gz")

# List all metabolites
for met in model.metabolites:
    print(met.id, met.name)

print(len(model.metabolites))

In [49]:
# Understanding effects on flux
for reaction in model.metabolites.atp_c.reactions:
    print(reaction, reaction.name)

BIOMASS_Ecoli_core_w_GAM: 1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c Biomass Objective Function with GAM
SUCOAS: atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c Succinyl-CoA synthetase (ADP-forming)
PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c Phosphoenolpyruvate carboxykinase
PPS: atp_c + h2o_c + pyr_c --> amp_c + 2.0 h_c + pep_c + pi_c Phosphoenolpyruvate synthase
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c Phosphofructokinase
GLNS: atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c Glutamine synthetase
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c ATP maintenance requirement
ADK1: amp_c + atp_c <=> 2.0 adp_c Adenylate kinase
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c Pyruvate ki

In [68]:
from cobra import Reaction, Metabolite
model.objective = model.reactions.BIOMASS_Ecoli_core_w_GAM


with model:
    # function name = (function)(('title'))
    magic_reaction = Reaction('magic')

    # Creating the reaction, and defining stoichiometry
    magic_reaction.add_metabolites({model.metabolites.g6p_c: -1, model.metabolites.atp_c: 55})

    # printing the reaction as a string
    print(magic_reaction.build_reaction_string())

    # adding bounds to make the rxn irreversible (or force the reaction pathway)
    magic_reaction.lower_bound = 1   # Reaction must carry at least 10 units of flux
    magic_reaction.upper_bound = 100 # Reaction can carry up to 100

    # even with these bounds and an extreme yield (1 -> 55) on this rxn provided, the cells don't 
    # need this pathway, and it is actually and unfavorable alternative to the methods it \
    # it is currently employing 

    #Adding the reaction to the model
    model.add_reactions([magic_reaction])

    # Since this is within the (with model) statement, this has at least temporarily added /
    # added the rxn to the model
    model.reactions.magic

    
    model.add_boundary(model.metabolites.atp_c, type='demand')

    Magic_Model = model.optimize()
    print(Magic_Model.fluxes['BIOMASS_Ecoli_core_w_GAM'])


Original_Flux = model.optimize()
print(Original_Flux.fluxes['BIOMASS_Ecoli_core_w_GAM'])

g6p_c --> 55 atp_c
0.775891153206166
0.8739215069684334


Time left? Add a pathway of your choice and optimize for flux through it!